In [1]:
import torch
import os

In [2]:
def calculate_entropy(pred_block):
    # Calculate the entropy for a given block
    entropy = -torch.sum(pred_block * torch.log(pred_block + 1e-10), dim=0)
    return entropy

def upc(label_image, unlabel_image, pred, label, N, k):
    _, _, h, w = pred.shape
    G = torch.zeros(N * N)
    h_new, w_new = h // N, w // N
    pred_soft = torch.softmax(pred, dim=1)
    print(pred_soft.shape)
    pred_argmax = torch.argmax(pred_soft,dim=1)
    mask = torch.zeros((h, w), dtype=torch.bool).unsqueeze(0)
    p = 0
    for i in range(N):
        for j in range(N): 
            block = pred_soft[:, N*i:N*(i+1), N*j:N*(j+1)]
            
            entropy = calculate_entropy(block)

            G[p] = entropy.mean().item()
            p += 1
    
    _, topk_idx = torch.topk(G, k)
    
    for idx in topk_idx:
        row = idx // N
        col = idx % N
        mask[:, row*h_new:(row+1)*h_new, col*w_new:(col+1)*w_new] = True
    
    mask_F = ~mask
    pesudo = pred_argmax * mask_F + label * mask
    unlabel_image = unlabel_image * mask_F + label_image * mask
    
    return unlabel_image, pesudo    
    

In [3]:
label_image = torch.rand(2,3,512,512)
unlabel_image = torch.rand(2,3,512,512)
pred = torch.rand(2,53,512,512)
label = torch.rand(2,512,512)
k = 3
N = 16
unlabel_image, psesudo = upc(label_image, unlabel_image, pred, label, N, k)
print(unlabel_image.shape)
print(psesudo.shape)

torch.Size([2, 3, 512, 512])
torch.Size([2, 512, 512])
